In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

!pip install pip3-autoremove -q
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124 -q
!pip install unsloth -q
!pip install --upgrade transformers==4.52.4 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 40.3 MB/s eta 0:00:00:00:0100:01


In [3]:
# !pip install -q unsloth

In [4]:
# unsloth.__version__ 2025.6.3
# torch.__version__ 2.7.0+cu126
# transformers.__version__ 4.51.3

In [2]:
import datasets
import numpy as np
import unsloth
import torch
import trl 
import transformers
import unsloth

print("datasets.__version__", datasets.__version__)
print("numpy.__version__", np.__version__)
print("unsloth.__version__", unsloth.__version__)
print("torch.__version__", torch.__version__)
print("transformers.__version__", transformers.__version__)
print("trl.__version__", trl.__version__)

# datasets.__version__ 3.6.0
# numpy.__version__ 1.26.4
# unsloth.__version__ 2025.6.3
# torch.__version__ 2.7.0+cu126
# transformers.__version__ 4.51.3
# trl.__version__ 0.19.0

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-30 11:12:34.293119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751281954.642221      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751281954.741210      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
datasets.__version__ 3.6.0
numpy.__version__ 1.26.4
unsloth.__version__ 2025.6.8
torch.__version__ 2.6.0+cu124
transformers.__version__ 4.52.4
trl.__version__ 0.19.0


In [2]:
import numpy as np
from datasets import concatenate_datasets, load_dataset

# from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only  
import torch

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

from unsloth import FastModel
from unsloth.chat_templates import get_chat_template

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-01 00:45:57.117936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751330757.494502      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751330757.608435      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
dataset_id = "tmnam20/ViMedAQA"
subset = "disease"
dataset = load_dataset(dataset_id, subset)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/501k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/478k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14121 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/784 [00:00<?, ? examples/s]

In [4]:
# Merge training and validation samples
test_samples = concatenate_datasets([dataset['validation'],
                                         dataset['test']])

# Shuffle the merged dataset with a fixed random seed
training_samples = dataset['train']
test_samples = test_samples.shuffle(seed=42)

print("Number of training samples:", training_samples.num_rows)
print("Number of test samples:", test_samples.num_rows)

Number of training samples: 14121
Number of test samples: 1569


In [9]:
dataset['train'][0]

{'question_idx': 'disease_13345',
 'question': 'Khi soi bằng đèn thường, triệu chứng điển hình của viêm mống mắt là gì?',
 'answer': 'Đồng tử 2 bên không đều nhau, đồng tử bên bệnh nhỏ hơn bên bình thường.',
 'context': '5.1 Nhìn về mặt đại thể (khi soi bằng đèn thường) - Đồng tử 2 bên không đều nhau (bên bệnh nhỏ hơn) Có thể phát hiện đồng tử co bên bệnh lý co nhiều hơn bên bình thường, điều này có thể do viêm nhiều dẫn đến dính thành sau mống mắt lại với nhau và với thuỷ tinh thể. Thông thường kích thường đồng tử là 2 – 3 mm, đều 2 bên. - Mủ ở đáy mống mắt (mủ tiền phòng) Đây là hiện tượng các tế bào viêm cùng với chất tiết (chứa nhiều fibrin). Mủ nằm ở dưới là do nguyên nhân trọng lực. 5.2 Khám mắt qua đèn khe: (split lamp) Đây là công cụ chuyên dụng của bác sĩ chuyên khoa Mắt. Qua đèn khe, bác sĩ có thể phát hiện: - Các tế bào nằm giữa giác mạc và mống mắt Số lượng tế bào này trên diện tích 1 mm² giúp phân độ nặng của viêm màng bồ đào. - Độ mờ thuỷ dịch Như đã miêu tả, thuỷ dịch là

In [ ]:
# To temporary Model hub
from huggingface_hub import HfApi
from huggingface_hub import login
# Initialize API
login("")
api = HfApi()

In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import snapshot_download
import os
api = HfApi()

repo_id = "danhtran2mind/Gemma-3-1B-Instruct-Vi-Medical-LoRA"
save_path = "Gemma-3-1B-Instruct-Vi-Medical-LoRA"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Download the dataset
snapshot_download(repo_id=repo_id, repo_type="model", local_dir=save_path)

Fetching 100 files:   0%|          | 0/100 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

adapter_config.json: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

trainer_state.json: 0.00B [00:00, ?B/s]

'/kaggle/working/Gemma-3-1B-Instruct-Vi-Medical-LoRA'

In [7]:
from unsloth import FastLanguageModel
import torch

model_id = "google/gemma-3-1b-it"

# Load the model and tokenizer
model, tokenizer = FastModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 16,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 42,
)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

==((====))==  Unsloth 2025.6.9: Fast Gemma3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Unsloth: Making `model.base_model.model.model` require gradients


In [8]:
def convert_squad_sample_to_conversation(sample):
    # get the question and context for this sample
    question = sample['question']
    context = sample['context']

    answers = sample['answer']
    if len(answers) == 0 :
      answer = "The context does not provide an answer..."
    else:
      answer = sample['answer']

    # now we define an initial model prompt defining the task and giving the model the context passage
    instruction_prompt_template = '''Bạn là một trợ lý hữu ích được giao nhiệm vụ trích xuất các đoạn văn trả lời câu hỏi của người dùng từ một ngữ cảnh cho trước. Xuất ra các đoạn văn chính xác từng từ một trả lời câu hỏi của người dùng. Không xuất ra bất kỳ văn bản nào khác ngoài các đoạn văn trong ngữ cảnh. Xuất ra lượng tối thiểu để trả lời câu hỏi, ví dụ chỉ 2-3 từ từ đoạn văn. Nếu không thể tìm thấy câu trả lời trong ngữ cảnh, xuất ra 'Ngữ cảnh không cung cấp câu trả lời...'

    Ngữ cảnh: {context}'''

    # now we'll convert these into a list of messages for our conversation
    messages = [
        {"role": "system", "content": instruction_prompt_template.format(context=context)},
        {"role": "user", "content": question},
        {"role": "assistant", "content": answer}
    ]
    
    sample_conversation = tokenizer.apply_chat_template(messages, tokenize=False)
    return {"text": sample_conversation, "messages": messages, "answer": answer}


In [9]:
conversation_training_samples = training_samples.map(convert_squad_sample_to_conversation)
conversation_test_samples = test_samples.map(convert_squad_sample_to_conversation)

Map:   0%|          | 0/14121 [00:00<?, ? examples/s]

Map:   0%|          | 0/1569 [00:00<?, ? examples/s]

In [14]:
conversation_test_samples[0]['answer']

'Nếu bạn có các triệu chứng nghi ngờ của nhiễm nấm da chân, hãy liên hệ với bác sĩ ngay để được tham vấn điều trị sớm. Điều này sẽ giúp bạn kiểm soát bệnh tình và ngăn ngừa các biến chứng nghiêm trọng.'

In [15]:
conversation_test_samples[0]['messages']

[{'content': "Bạn là một trợ lý hữu ích được giao nhiệm vụ trích xuất các đoạn văn trả lời câu hỏi của người dùng từ một ngữ cảnh cho trước. Xuất ra các đoạn văn chính xác từng từ một trả lời câu hỏi của người dùng. Không xuất ra bất kỳ văn bản nào khác ngoài các đoạn văn trong ngữ cảnh. Xuất ra lượng tối thiểu để trả lời câu hỏi, ví dụ chỉ 2-3 từ từ đoạn văn. Nếu không thể tìm thấy câu trả lời trong ngữ cảnh, xuất ra 'Ngữ cảnh không cung cấp câu trả lời...'\n\n    Ngữ cảnh: Những mẹo này có thể giúp bạn tránh bị nấm da chân hoặc giảm bớt các triệu chứng nếu nhiễm nấm xảy ra: - Giữ chân khô ráo, đặc biệt là giữa các ngón chân. Đi chân trần để chân thoát khí nhiều nhất có thể khi ở nhà. Lau khô các kẽ ngón chân sau khi tắm.\n- Thay tất thường xuyên. Nếu chân bạn ra nhiều mồ hôi, hãy thay tất hai lần một ngày.\n- Đi giày nhẹ, thông thoáng. Tránh giày làm bằng chất liệu tổng hợp, chẳng hạn như nhựa vinyl hoặc cao su.\n- Mang thay đổi các đôi giày. Đừng mang cùng một đôi mỗi ngày để giày c

In [16]:
conversation_test_samples[0]['text']

"<bos><start_of_turn>user\nBạn là một trợ lý hữu ích được giao nhiệm vụ trích xuất các đoạn văn trả lời câu hỏi của người dùng từ một ngữ cảnh cho trước. Xuất ra các đoạn văn chính xác từng từ một trả lời câu hỏi của người dùng. Không xuất ra bất kỳ văn bản nào khác ngoài các đoạn văn trong ngữ cảnh. Xuất ra lượng tối thiểu để trả lời câu hỏi, ví dụ chỉ 2-3 từ từ đoạn văn. Nếu không thể tìm thấy câu trả lời trong ngữ cảnh, xuất ra 'Ngữ cảnh không cung cấp câu trả lời...'\n\n    Ngữ cảnh: Những mẹo này có thể giúp bạn tránh bị nấm da chân hoặc giảm bớt các triệu chứng nếu nhiễm nấm xảy ra: - Giữ chân khô ráo, đặc biệt là giữa các ngón chân. Đi chân trần để chân thoát khí nhiều nhất có thể khi ở nhà. Lau khô các kẽ ngón chân sau khi tắm.\n- Thay tất thường xuyên. Nếu chân bạn ra nhiều mồ hôi, hãy thay tất hai lần một ngày.\n- Đi giày nhẹ, thông thoáng. Tránh giày làm bằng chất liệu tổng hợp, chẳng hạn như nhựa vinyl hoặc cao su.\n- Mang thay đổi các đôi giày. Đừng mang cùng một đôi mỗi n

In [ ]:
arguments = TrainingArguments(  
                resume_from_checkpoint="./Gemma-3-1B-Instruct-Vi-Medical-LoRA",
                per_device_train_batch_size=4,  
                per_device_eval_batch_size=4,  
                gradient_accumulation_steps=4,
                eval_strategy="steps",
                eval_steps=200,
                logging_steps=50,
                save_steps=200,
                warmup_steps=30,
                save_total_limit=4,
                num_train_epochs=7, # 6
                # max_steps=60,
                save_strategy="steps",
                metric_for_best_model="eval_loss",
                learning_rate=2e-4,  
                fp16=not is_bfloat16_supported(),  
                bf16=is_bfloat16_supported(),  
                optim="adamw_8bit",  
                weight_decay=0.01,  
                lr_scheduler_type="linear",  
                seed=42,  
                output_dir="Gemma-3-1B-Instruct-Vi-Medical-LoRA ",  
                report_to="none",
        		load_best_model_at_end=True,  # Load weights with lowest val loss
        		greater_is_better=False,
            )

In [14]:
# Define Trainer
trainer = SFTTrainer(  
    model=model,  
    tokenizer=tokenizer,  
    train_dataset=conversation_training_samples,  
    eval_dataset=conversation_test_samples,  
    # dataset_text_field="text",
    max_seq_length=2048,  
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  
    dataset_num_proc=2,  
    packing=False,  # Can make training 5x faster for short sequences.  
    args=arguments,
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"]:   0%|          | 0/14121 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/1569 [00:00<?, ? examples/s]

In [15]:
# Create train_on_responses_only
trainer = train_on_responses_only(  
    trainer,  
    instruction_part="<start_of_turn>user\n",  
    response_part="<start_of_turn>model\n",  
)

Map (num_proc=4):   0%|          | 0/14121 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1569 [00:00<?, ? examples/s]

Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [16]:
# Start train process
trainer.train(resume_from_checkpoint=True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 14,121 | Num Epochs = 7 | Total steps = 6,181
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 13,045,760 of 1,000,000,000 (1.30% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
2800,0.441800,0.609476
3000,0.463900,0.612166
3200,0.480000,0.604576
3400,0.458700,0.599432
3600,0.356500,0.623287
3800,0.374500,0.633959
4000,0.401900,0.629742
4200,0.404300,0.632301
4400,0.373600,0.630397
4600,0.296100,0.672576


Could not locate the best model at Gemma-3-1B-Instruct-Vi-Medical-LoRA/checkpoint-800/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=6181, training_loss=0.20157300330465847, metrics={'train_runtime': 18291.2443, 'train_samples_per_second': 5.404, 'train_steps_per_second': 0.338, 'total_flos': 2.0813416244964864e+17, 'train_loss': 0.20157300330465847})

In [ ]:
model.save_pretrained("Gemma-3-1B-Instruct-Vi-Medical-LoRA")  # Local saving
tokenizer.save_pretrained("Gemma-3-1B-Instruct-Vi-Medical-LoRA ")


('Gemma-3-1B-Instruct-Vi-Medical-LoRA/tokenizer_config.json',
 'Gemma-3-1B-Instruct-Vi-Medical-LoRA/special_tokens_map.json',
 'Gemma-3-1B-Instruct-Vi-Medical-LoRA/chat_template.jinja',
 'Gemma-3-1B-Instruct-Vi-Medical-LoRA/tokenizer.model',
 'Gemma-3-1B-Instruct-Vi-Medical-LoRA/added_tokens.json',
 'Gemma-3-1B-Instruct-Vi-Medical-LoRA/tokenizer.json')

# Inference

In [ ]:
# Define the question
question = ("Khi nghi ngờ bị loét dạ dày tá tràng nên đến khoa nào "
          "tại bệnh viện để thăm khám?")

# Create text generation pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map=device,
    max_new_tokens=2048,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    top_k=64,
)

# Format input for the pipeline
input_prompt = [{"role": "user", "content": question}]

# Generate response
output = generator(input_prompt, return_full_text=False)[0]

# Print the generated text
print(output["generated_text"])